In [1]:
import datasets

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")


/opt/homebrew/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [5]:
#reference dataset to match styling
import json


with open("data/train.json") as f:
    data = json.load(f)

In [7]:
len(data)

6807

In [153]:
data[0].keys()

dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])

In [10]:
#document is rand id
#full_text is simply the text
#tokens is full text split by tokens and put into list
#traling white space is a boolean saying whether there is a space after that token
#labels is 0 if NA or label if it has a label

In [38]:
#up to 29908 are english

In [171]:
document_ids = dataset['train']['id'][:29908]

In [172]:
full_text = dataset['train']['source_text'][:29908]

In [173]:
span_labels = dataset['train']['span_labels'][:29908]

In [174]:
from ast import literal_eval

label_text = []
labels = set()
for i, spans in enumerate(span_labels):
    spans = literal_eval(spans)
    text = full_text[i]
    for span in spans:
        labels.add(span[2])
        text = text[:span[0]] + span[2] + text[span[1]+1:]
    label_text.append(text)

In [175]:
#let's use the spacy english tokenizer
# Construction 2
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

In [176]:
tokens = [[str(i) for i in tokenizer(i)] for i in full_text]

In [177]:
labels = list(labels)

In [178]:
label_tokens = [[str(i) for i in tokenizer(i)] for i in label_text]

In [179]:
for label in label_tokens:
    for i,j in enumerate(label):
        label[i] = '0'
        for l in labels:
            if l in j:
                label[i] = l
                break


In [180]:
data = []

for i in range(len(document_ids)):
    data.append({
        'document': document_ids[i],
        'full_text': full_text[i],
        'tokens': tokens[i],
        'labels': label_tokens[i]
    })
